# Instalando a biblioteca do CNPJ



In [ ]:
pip install basedosdados


In [ ]:
pip install --upgrade pyarrow

  Using cached pyarrow-17.0.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (3.3 kB)
Using cached pyarrow-17.0.0-cp310-cp310-manylinux_2_28_x86_64.whl (39.9 MB)
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 6.0.0
    Uninstalling pyarrow-6.0.0:
      Successfully uninstalled pyarrow-6.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
basedosdados 1.6.11 requires pyarrow==6.0.0, but you have pyarrow 17.0.0 which is incompatible.
bigframes 1.17.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.16.0, but you have google-cloud-bigquery 2.30.1 which is incompatible.
bigframes 1.17.0 requires google-cloud-storage>=2.0.0, but you have google-cloud-storage 1.42.3 which is incompatible.
bigquery-magics 0.2.0 requires google-cloud-bigquery<4.0.0dev,>=3.13.0, but you have google-cloud-bigquery 2.30.1 which is incompatible.
cudf-cu12 24.4.

# Importando as bibliotecas


In [ ]:
import pandas as pd
import numpy as np
from pandas import DataFrame
import seaborn as sns
from google.colab import drive
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import graphviz
from google.cloud import bigquery
import pandas as pd
import basedosdados as bd


AttributeError: module 'shapely' has no attribute 'strtree'

# Pegando a base de dados de alunos do BSI

In [ ]:
drive.mount('/content/gdrive/')

In [ ]:
#data_evasao_desempenho = pd.read_excel("/content/gdrive/My Drive/EvasaoBSI.xlsx")


In [ ]:
#data_cpf = pd.read_excel("/content/gdrive/My Drive/EgressosBaseUnirio.xlsx"


In [ ]:
file_name = "Cr_BSI_1"
#file_name = "CrMat_1"
#file_name = "CrEngProd_1"
data_cpf = pd.read_excel("/content/gdrive/My Drive/dados_mestrado/"+file_name+".xlsx")

In [ ]:
# prompt: Substitua na coluna NOME de data_cpf substrings ' por espaço

data_cpf['NOME_PESSOA'] = data_cpf['NOME_PESSOA'].str.replace("'", " ")


# Tratando o CPF

In [ ]:
'''
# No valor de data_cpf['CPF_TRATADO'] que tiver 10 caracteres, adicione um 0 na frente.
data_cpf['CPF'] = data_cpf['CPF'].astype(str)
for i in range(10, 0, -1):
    data_cpf['CPF'] = np.where(data_cpf['CPF'].str.len() == i, '0' * (11 - i) + data_cpf['CPF'], data_cpf['CPF'])
'''

In [ ]:
# Crie uma nova coluna em data_cpf chamada  CPF_TRATADO, que é formada do seguinte modo: substitua os três primeiros algoritmos por * e substitua os dois últimos algoritmos por * também. Por exemplo: 12345678911 vira ***456789**
'''
data_cpf['CPF_TRATADO'] = data_cpf['CPF'].astype(str).str.replace(r'(\d{3})(\d{3})(\d{3})(\d{2})', r'***\2\3**')
'''

In [ ]:

data_cpf['CPF_TRATADO']

# Descobrindo os egressos que são donos de empresas

In [ ]:
# Cria uma lista com os nomes únicos
nomes = data_cpf['NOME_PESSOA'].unique().tolist()
cpfs = data_cpf['CPF_TRATADO'].unique().tolist()

# Cria uma string para a cláusula WHERE com os nomes
nomes_sql = "', '".join(nomes)
cpfs_sql = "', '".join(cpfs )

# Cria o cliente BigQuery
client = bigquery.Client(project='dissertacao-416315')

# Define a consulta com a cláusula WHERE para filtrar pelos nomes
query = f"""
SELECT * FROM `basedosdados.br_me_cnpj.socios`
WHERE nome IN ('{nomes_sql}') and documento in ('{cpfs_sql}')
"""

# Executa a consulta e escreve o resultado em um DataFrame
df_socios = pd.read_gbq(query, project_id='dissertacao-416315', reauth=True)

In [ ]:
'''
nomes = data_cpf['NOME_PESSOA'].unique().tolist()
cpfs = data_cpf['CPF_TRATADO'].unique().tolist()

client = bigquery.Client(project='dissertacao-416315')

# Defina a consulta
query = """
SELECT * FROM `basedosdados.br_me_cnpj.Simples` x join `basedosdados.br_me_cnpj.socios` y on (x.cnpj_basico = y.cnpj_basico and  y.nome IN ('{nomes_sql}') and y.documento in ('{cpfs_sql}' ))
"""

# Execute a consulta e escreva o resultado em um DataFrame
df_meis = pd.read_gbq(query, project_id='dissertacao-416315', reauth=True)
'''

In [ ]:
# prompt: Crie uma nova coluna em df_socios chamada 'mei' onde houver intersecção de 'nome' de df_socios com 'nome' com df_meis
'''
df_socios['mei'] = df_socios['nome'].isin(df_meis['nome']).astype(int)
df_socios
'''

In [ ]:
# prompt: Pegue valores únicos de df_socios pelo nome

df_socios_unicos = df_socios.drop_duplicates(subset='nome')
#df_socios_unicos

In [ ]:
# prompt: Crie uma nova coluna em data_cpf chamada "Empresario". Preencha essa coluna com 1 caso data_cpf.NOME = df_socios_unicos.nome e data_cpf.CPF_TRATADO = df_socios_unicos.documento, caso o contrário, preencha com 0.

data_cpf['Empresario'] = np.where((data_cpf['NOME_PESSOA'].isin(df_socios_unicos['nome'])) & (data_cpf['CPF_TRATADO'].isin(df_socios_unicos['documento'])), 1, 0)


In [ ]:
#data_cpf

In [ ]:
# prompt: Liste os valores das colunas de data_cpf cujo Empresario = 1.

#data_cpf[data_cpf['Empresario'] == 1]


In [ ]:
# prompt: Liste os valores das colunas de data_cpf cujo Empresario = 1 e CURSO = "Sistemas de Informação - Bacharelado - Turno Integral (V/N)"

#data_cpf[(data_cpf['Empresario'] == 1) & (data_cpf['NOME_CURSO'] == "Sistemas de Informação - Bacharelado - Turno Integral (V/N)")]


In [ ]:
# prompt: Faça um histograma dos 10 CURSO que tem mais empresários = 1

data_cpf.groupby('NOME_CURSO')['Empresario'].sum().nlargest(10).plot(kind='bar')


# Trabalhando com as datas de evasão e entrada na sociedade

In [ ]:
data_cpf['DT_EVASAO']

In [ ]:
# prompt: Junte df_socios com data_cpf onde df_socios.nome = data_cpf.NOME e df_socios.documento =  data_cpf.CPF_TRATADO

df_merged = pd.merge(df_socios_unicos, data_cpf, left_on=['nome', 'documento'], right_on=['NOME_PESSOA', 'CPF_TRATADO'])


In [ ]:
df_merged

In [ ]:
df_merged['data_entrada_sociedade']

In [ ]:
df_merged['DT_EVASAO']

In [ ]:
# prompt: Converta df_merged['data_entrada_sociedade'] para datetime64[ns]

df_merged['data_entrada_sociedade'] = pd.to_datetime(df_merged['data_entrada_sociedade'], errors='coerce', format='%Y-%m-%d')
df_merged['DT_EVASAO'] = pd.to_datetime(df_merged['DT_EVASAO'], errors='coerce', format='%Y-%m-%d')
df_merged['year4'] = pd.to_datetime(df_merged['year4'], errors='coerce', format='%Y-%m-%d')

In [ ]:
# df_merged['DT_EVASAO']

In [ ]:
# df_merged['data_entrada_sociedade']

In [ ]:
# prompt: Crie uma nova coluna em df_merged chamada Tempo_Evasao_Sociedade, que é a diferença de tempo entre DT_EVASAO e data_entrada_sociedade

df_merged['Tempo_Evasao_Sociedade'] = df_merged['DT_EVASAO'] - df_merged['data_entrada_sociedade']
df_merged['Tempo_Evasao_Sociedade']

In [ ]:
#df_merged['Tempo_Evasao_Sociedade'] = df_merged['year4'] - df_merged['data_entrada_sociedade']
#df_merged['Tempo_Evasao_Sociedade']

In [ ]:
# prompt: Converta df_merged['ANO_INGRESSO'] ] para DatetimeArray

df_merged['ANO_INGRESSO'] = pd.to_datetime(df_merged['ANO_INGRESSO'], format='%Y')


In [ ]:
df_merged['Tempo_Universidade_Sociedade'] = df_merged['ANO_INGRESSO']  - df_merged['data_entrada_sociedade']
df_merged['Tempo_Universidade_Sociedade']

In [ ]:
# prompt: Faça um gráfico de barras de df_merged['Tempo_Evasao_Sociedade'] para duas categoria: valores menores que 0 e valores maiores que 0.

df_merged['Tempo_Evasao_Sociedade_Cat'] = np.where(df_merged['Tempo_Evasao_Sociedade'] < pd.Timedelta(0), 'Fundou uma empresa depois da graduação' ,'Fundou uma empresa antes da graduação')

df_merged.groupby('Tempo_Evasao_Sociedade_Cat')['Tempo_Evasao_Sociedade'].count().plot(kind='bar')


In [ ]:
df_merged['Tempo_Universidade_Sociedade_Cat'] = np.where(df_merged['Tempo_Universidade_Sociedade'] < pd.Timedelta(0),'Fundou uma empresa depois de entrar na faculdade' , 'Fundou uma empresa antes de entrar na faculdade')

df_merged.groupby('Tempo_Universidade_Sociedade_Cat')['Tempo_Universidade_Sociedade'].count().plot(kind='bar')

In [ ]:
# prompt: Coloque a tabela df_merged['Tempo_Evasao_Sociedade_Cat'] em data_cpf onde df_socios.nome = data_cpf.NOME e df_socios.documento =  data_cpf.CPF_TRATADO

data_cpf = data_cpf.merge(df_merged[['Tempo_Evasao_Sociedade_Cat']], left_on=['NOME_PESSOA', 'CPF_TRATADO'], right_on=[df_merged['nome'],df_merged['documento'] ], how='left')
data_cpf['Tempo_Evasao_Sociedade_Cat'].fillna('Não é empresário', inplace=True)
# data_cpf

In [ ]:
data_cpf = data_cpf.merge(df_merged[['Tempo_Universidade_Sociedade_Cat']], left_on=['NOME_PESSOA', 'CPF_TRATADO'], right_on=[df_merged['nome'],df_merged['documento'] ], how='left')
data_cpf['Tempo_Universidade_Sociedade_Cat'].fillna('Não é empresário', inplace=True)

# Consolidação para o modelo

In [ ]:
#data_cpf

In [ ]:
# data_cpf.to_excel("/content/gdrive/My Drive/dados_mestrado/data_cpf_modelo.xlsx")

In [ ]:
#data_cpf[data_cpf['Empresario'] == 1]


In [ ]:
def definir_categoria(row):
    if row['Tempo_Evasao_Sociedade_Cat'] == 'Não é empresário' and row['Tempo_Universidade_Sociedade_Cat'] == 'Não é empresário':
        return 'Não é empresário'
    elif row['Tempo_Evasao_Sociedade_Cat'] == 'Fundou uma empresa antes da graduação' and row['Tempo_Universidade_Sociedade_Cat'] == 'Fundou uma empresa depois de entrar na faculdade':
        return 'Fundou uma empresa durante a graduação'
    elif row['Tempo_Evasao_Sociedade_Cat'] == 'Fundou uma empresa depois da graduação':
        return 'Fundou uma empresa depois da graduação'
    elif row['Tempo_Universidade_Sociedade_Cat'] == 'Fundou uma empresa antes de entrar na faculdade':
        return 'Fundou uma empresa antes de entrar na graduação'
    else:
        return 'Categoria Não Definida'  # Para qualquer caso que não se encaixe nas condições acima


# Aplicar a função para criar a nova coluna
data_cpf['Categoria'] = data_cpf.apply(definir_categoria, axis=1)


In [ ]:
# prompt: Faça de um gráfico de barras de Tempo_Evasao_Sociedade_Cat de data_cpf
data_cpf_unicos = data_cpf.drop_duplicates(subset='NOME_PESSOA')

data_cpf_unicos['Categoria'].value_counts().plot(kind='bar')


In [ ]:
data_cpf_unicos[data_cpf_unicos['Empresario'] == 1]

In [ ]:
data_iguais = data_cpf[data_cpf['ANO'] == data_cpf['ANO_INGRESSO']]
data_unicos = data_iguais.drop_duplicates(subset='NOME_PESSOA')


file_name = file_name+"_2.xlsx"
data_cpf_unicos.to_excel("/content/gdrive/My Drive/dados_mestrado/"+file_name)

In [ ]:
file_name = file_name+"_2.xlsx"
data_cpf_unicos.to_csv("/content/gdrive/My Drive/dados_mestrado/"+file_name)

In [ ]:
# Reduce the number of columns by dropping unnecessary columns
#data_cpf_reduced = data_cpf.drop(columns=['Tempo_Evasao_Sociedade_Cat', 'Tempo_Universidade_Sociedade_Cat', 'NOME_CURSO', 'CPF_TRATADO', 'NOME_PESSOA', 'DT_EVASAO', 'FORMA_EVASAO'])

# Save the reduced DataFrame to Excel
#data_cpf_reduced.to_excel("/content/gdrive/My Drive/dados_mestrado/data_cpf_modelo_bsi_notas.xlsx")

In [ ]:
'''
data_bsi = data_cpf[(data_cpf['Empresario'] == 1) & (data_cpf['NOME_CURSO'] == "Sistemas de Informação - Bacharelado - Turno Integral (V/N)")]

# Cria uma lista com os nomes únicos
nomes_bsi = data_bsi['NOME_PESSOA'].unique().tolist()
cpfs_bsi = data_bsi['CPF_TRATADO'].unique().tolist()

# Cria uma string para a cláusula WHERE com os nomes
nomes_sql_bsi = "', '".join(nomes_bsi)
cpfs_sql_bsi = "', '".join(cpfs_bsi)

# Crie um cliente BigQuery
client = bigquery.Client(project='dissertacao-416315')

# Defina a consulta
query = f"""
SELECT * FROM `basedosdados.br_me_cnpj.empresas` x
join `basedosdados.br_me_cnpj.socios` y on (x.cnpj_basico = y.cnpj_basico
and y.nome in ('{nomes_sql_bsi}') and documento in ('{cpfs_sql_bsi}')) LIMIT 1000
"""
# Execute a consulta e escreva o resultado em um DataFrame
df_donos_bsi = pd.read_gbq(query, project_id='dissertacao-416315', reauth=True)
'''

In [ ]:
# df_donos_bsi = df_donos_bsi.drop_duplicates(subset='nome')
# df_donos_bsi

In [ ]:
# prompt: Crie um dataset chamado de empresarios_bsi juntado df_donos_bsi.nome com data_cpf.NOME  e df_donos_bsi.documento com data_cpf.CPF_TRATADO. Pegue as somente essas colunas e df_donos_bsi.razao_social e data_cpf.Tempo_Evasao_Sociedade_Cat

# empresarios_bsi = pd.merge(df_donos_bsi[['nome', 'documento', 'razao_social']], data_cpf[['NOME_PESSOA', 'CPF_TRATADO', 'Tempo_Evasao_Sociedade_Cat']], left_on=['nome', 'documento'], right_on=['NOME_PESSOA', 'CPF_TRATADO'])[['nome', 'documento', 'razao_social', 'Tempo_Evasao_Sociedade_Cat']]
# empresarios_bsi

In [ ]:
# prompt: Pegue empresarios_bsi por nome único

# empresarios_bsi_unicos = empresarios_bsi.drop_duplicates(subset='nome')
# empresarios_bsi_unicos

# Modelo para prever desempenho acadêmico

In [ ]:
data_modelo = data_cpf[['ingresso_atual', 'MEDIA_FINAL', 'CR_ATUAL', 'CR_NO_PERIODO', 'Empresario', 'Categoria']]


In [ ]:
# Transformar as colunas de data modelo que são categóricas, como ingresso_atual, FORMA_EVASAO e Tempo_Evasao_Sociedade_Cat, em colunas numéricas
from sklearn.preprocessing import LabelEncoder

# Create a LabelEncoder object
le = LabelEncoder()

# Apply the label encoder to the categorical columns
data_modelo['ingresso_atual'] = le.fit_transform(data_modelo['ingresso_atual'])
data_modelo['Categoria'] = le.fit_transform(data_modelo['Categoria'])

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)

X_resampled, y_resampled = smote.fit_resample(data_modelo.drop("Empresario", axis=1), data_modelo["Empresario"])

data_resampled = pd.concat([pd.DataFrame(X_resampled, columns=data_modelo.drop("Empresario", axis=1).columns), pd.DataFrame(y_resampled, columns=["Empresario"])], axis=1)

data_resampled.groupby("Empresario").size().plot(kind="bar")

In [ ]:
'''
# Define the dependent and independent variables
dependent_variable = data_modelo["CR_ATUAL"]
independent_vars = data_modelo[["ingresso_atual", "MEDIA_FINAL", "CR_NO_PERIODO", "Empresario", "Categoria"]]
'''


In [ ]:
'''
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(independent_vars, dependent_variable)

# Coeficientes da regressão linear
coefficients = model.coef_

# Calcular o valor agregado das variáveis independentes
aggregated_value = np.dot(independent_vars, coefficients)

# Prever os valores da variável dependente usando o modelo ajustado
predicted_values = model.predict(independent_vars)

# Criar o scatterplot
plt.scatter(aggregated_value, dependent_variable, label='Real Data')

# Plotar a linha de regressão
# Note que usamos `aggregated_value` para criar a linha, mas poderíamos também usar as variáveis independentes diretamente
plt.plot(aggregated_value, predicted_values, color='red', label='Regression Line')

# Adicionar rótulos e título
plt.xlabel('Independent features: Agregated Value of admission method, final grade on each discipline, semester GPA, IsTheyBusinessperson, Category of businessperson student')
plt.ylabel('Dependent feature: Accumulated GPA')
plt.title('Scatter Plot of Dependent Variable and Added Value of Independent Variables with Regression Line')
plt.legend()

# Mostrar o gráfico
plt.show()
'''

In [ ]:
'''
r_squared = model.score(independent_vars, dependent_variable)
print(f'R^2 value: {r_squared}')
'''

In [ ]:
'''
from sklearn.model_selection import cross_val_score

cv_scores = cross_val_score(model, independent_vars, dependent_variable, cv=5, scoring='r2')
print(f'Scores de validação cruzada: {cv_scores}')
print(f'Média dos scores de validação cruzada: {cv_scores.mean()}')
'''